In [66]:
import json
from collections import defaultdict
import string
import zhconv
import sys,os,re
sys.path.append(os.path.abspath('../'))

In [209]:
from bart import MyBart
import numpy as np
from tqdm import tqdm
from utils.utils import read_data,read_data_nlpcc,read_kg
import torch
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('/home/xhsun/NLP/huggingfaceModels/Chinese/chinese-bart-base')
device=torch.device('cuda')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [20]:
model = MyBart.from_pretrained('/home/xhsun/NLP/huggingfaceModels/Chinese/chinese-bart-base',
                               state_dict=torch.load('/home/xhsun/Desktop/graduate_models/Section4/kgclue/best-model.pt',map_location='cpu'))
model.to(device)

MyBart(
  (model): BartModel(
    (shared): Embedding(21128, 768, padding_idx=0)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(21128, 768, padding_idx=0)
      (embed_positions): BartLearnedPositionalEmbedding(514, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): BartEncoderLayer(


In [35]:
def predict_ner(data,model,tokenizer):
    #ner_predictions=[]
    for example in tqdm(data,total=len(data),unit='question'):
        text=example['question'].lower().replace('“','').replace('”','').replace('"','').replace('"','').replace('\n','').replace('\t','')
        text='{}'.format(text)+'。上个句子中哪些属于实体名词？_。'#'你知道泰山极顶的作家是谁啊？。上个句子中哪些属于实体名词？_。'
        model_inputs=tokenizer([text],return_tensors='pt')
        input_ids=model_inputs['input_ids'].to(device)
        attention_mask=model_inputs['attention_mask'].to(device)
        outputs = model.generate(input_ids=input_ids,
                                attention_mask=attention_mask,
                                num_beams=4,
                                min_length=1,
                                max_length=30,
                                early_stopping=True,)#tensor([[ 102,  101, 3805, 2255, 3353, 7553,  102]], device='cuda:0')
        pred=tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=True).strip()#'泰 山 极 顶'
        example.update({"ner_pred":pred})
#         ner_predictions.append(example)
#     return ner_predictions

In [36]:
data=read_data('../../data/kgclue/test_public.json')

原始数据有2000个样本


In [37]:
data[0]

{'id': 0,
 'question': '你知道长江武汉航道局管多少个地方吗？',
 'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里'}

In [38]:
predict_ner(data,model,tokenizer)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [01:04<00:00, 30.98question/s]


In [39]:
data[:10]

[{'id': 0,
  'question': '你知道长江武汉航道局管多少个地方吗？',
  'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里',
  'ner_pred': '长 江 武 汉 航 道 局'},
 {'id': 1,
  'question': '东瓯王发生的主要事件是什么？',
  'answer': '东瓯王 ||| 主要事件 ||| 抗秦反秦，助汉击楚。',
  'ner_pred': '东 瓯 王'},
 {'id': 2,
  'question': '山西省铸造新工艺工程技术研究中心是什么的管辖的啊？',
  'answer': '山西省铸造新工艺工程技术研究中心 ||| 机构类型 ||| 研究开发机构',
  'ner_pred': '山 西 省 铸 造 新 工 艺 工 程 技 术 研 究 中 心'},
 {'id': 3,
  'question': '你知道谁是丘堤的儿媳吗',
  'answer': '丘堤 ||| 儿媳 ||| 籍虹',
  'ner_pred': '丘 堤'},
 {'id': 4,
  'question': '国民军参与过什么战争？',
  'answer': '国民军 ||| 参与战争 ||| 直奉战争、北伐战争、中原大战、抗日战争',
  'ner_pred': '国 民 军'},
 {'id': 5,
  'question': '你知道诺基亚lumia1320的主屏尺寸是多少吗？',
  'answer': '诺基亚lumia1320 ||| 主屏尺寸 ||| 6英寸',
  'ner_pred': '诺 基 亚 lumia1320'},
 {'id': 6,
  'question': '你什么时候参加的执业兽医资格考试啊？',
  'answer': '执业兽医资格考试 ||| 考试时间 ||| 10月',
  'ner_pred': '执 业 兽 医 资 格 考 试'},
 {'id': 7,
  'question': '你知道2007年度北京流行音乐典礼是哪种类型的奖项吗',
  'answer': '2007年度北京流行音乐典礼 ||| 奖项类型 ||| 音乐类奖项',
  'ner_pred': '2007 年 度 北 京 流 行 音 乐 典 礼'},
 

In [100]:
def select_span(tmp_q,question,ner_pred):
    start_id=tmp_q.find(ner_pred)
    end_id=start_id+len(ner_pred)
    new_pred=question[start_id:end_id]
    return new_pred

def rule_of_departure_to_destination(question,ner_pred):
    '''
    question='东营—深圳公路途径哪些城市'
    ner_pred=''东营深圳公路''
    '''
    ner_pred=''.join(ner_pred.split())
    try:
        
        if '—' in question:
            tmp_q=question.replace('—','')
            if ner_pred in tmp_q:
                start_token=ner_pred[:2]
                end_token=ner_pred[-2:]
                assert start_token in question and end_token in question
                start_pos=question.find(start_token)
                end_pos=question.find(end_token)+1
                entity=question[start_pos:end_pos+1]
                return entity
            else:
                return ''
        else:
            return ''
    
    except Exception as e:
        print(e)
        return ''

def rule_of_chinese_quotation_mark(question,ner_pred):
    '''
    question='请问“一带一路”铁路国际人才教育联盟是由哪个单位发起的？'
    ner_pred='一带一路铁路国际人才教育联盟'
    '''
    ner_pred=''.join(ner_pred.split())
    try:
        if '“' in question:
            assert '”' in question
            token_string=re.findall('“(.*)”',question)[0]
            tmp_q=question.replace('“','').replace('”','')
            if ner_pred in tmp_q:
                assert token_string in ner_pred
                new_pred=ner_pred.replace(token_string,'“{}”'.format(token_string))
                assert new_pred in question
                return new_pred
            else:
                return ''
        else:
            return ''
        
    except Exception as e:
        print(e)
        return ''
            
def rule_of_zhconv(question,ner_pred):
    '''
    question='中国共产党中央委员会政法委员会属于什么机构你知道吗？'
    ner_pred='中国共产党中央委员会政法委员會'
    '''
    ner_pred=''.join(ner_pred.split())
    if zhconv.convert(ner_pred,'zh-hans') in question:
        return zhconv.convert(ner_pred,'zh-hans')
    else:
        return ''
    
def final_rule(question,ner_pred):
    ner_pred=''.join(ner_pred.split())
    question_lower=question.lower()
    start_token=ner_pred[:2]
    end_token=ner_pred[-2:]
    
    if start_token in question_lower and end_token in question_lower:
        start_pos=question_lower.find(start_token)
        end_pos=question_lower.find(end_token)+1
        entity=question[start_pos:end_pos+1]
        return entity
    else:
        return ''

wrong_results=[]

for example in data:
    question=example['question']
    question_lower=question.lower()
    ner_pred=example['ner_pred']
    if ner_pred in question:
        continue
        
    elif ner_pred in question_lower:
        new_pred=select_span(tmp_q=question_lower,question=question,ner_pred=ner_pred)
        example.update({"ner_pred":new_pred})
        
    elif ''.join(ner_pred.split()) in question:
        new_pred=select_span(tmp_q=question,question=question,ner_pred=''.join(ner_pred.split()))
        example.update({"ner_pred":new_pred})
        
    elif ''.join(ner_pred.split()) in question_lower:
        new_pred=select_span(tmp_q=question_lower,question=question,ner_pred=''.join(ner_pred.split()))
        example.update({"ner_pred":new_pred})
        
    elif rule_of_departure_to_destination(question,ner_pred=ner_pred)!='':
        new_pred=rule_of_departure_to_destination(question,ner_pred=ner_pred)
        example.update({"ner_pred":new_pred})
    
    elif rule_of_chinese_quotation_mark(question,ner_pred=ner_pred)!='':
        new_pred=rule_of_chinese_quotation_mark(question,ner_pred=ner_pred)
        example.update({"ner_pred":new_pred})
    
    elif rule_of_zhconv(question,ner_pred=ner_pred)!='':
        new_pred=rule_of_zhconv(question,ner_pred=ner_pred)
        example.update({"ner_pred":new_pred})
        
    elif final_rule(question,ner_pred=ner_pred)!='':
        new_pred=final_rule(question,ner_pred=ner_pred)
        example.update({"ner_pred":new_pred})
        
    else:
        wrong_results.append(example)

In [101]:
len(wrong_results)

4

In [102]:
wrong_results

[{'id': 567,
  'question': '东北鼢鼠有什么天敌啊',
  'answer': '东北鼢鼠（哺乳纲动物） ||| 天敌 ||| 猫、猫头鹰和蛇等',
  'ner_pred': '东 北 鼠'},
 {'id': 659,
  'question': '于鋆炜的拦网高度是多少',
  'answer': '于鋆炜 ||| 拦网高度 ||| 300cm',
  'ner_pred': '于 炜'},
 {'id': 907,
  'question': '10·26朴正熙遇刺事件有哪些人员',
  'answer': '10·26朴正熙遇刺事件 ||| 主要人员 ||| 金载圭、朴正熙、车智澈',
  'ner_pred': '10 · 26 朴 正 熙 遇 刺 案'},
 {'id': 1529,
  'question': '请问慕容莎是什么时候出生的？',
  'answer': '慕容莎 ||| 出生年份 ||| 公元390年',
  'ner_pred': '请 容 莎'}]

In [219]:
alias_map,sub_map=read_kg(kg_path='/home/xhsun/NLP/KGQA/KG/nlpcc2018/knowledge/nlpcc-iccpol-2016.kbqa.kb',kg='nlpcc')

The number of triples: 43063796


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 43063796/43063796 [01:06<00:00, 652452.33it/s]


In [148]:
if '' in alias_map:
    del alias_map['']
if '' in sub_map:
    del sub_map['']
ner_data=[]
TP=0
FP=0
FN=0
FP_examples=[]
FN_examples=[]
with open('../outputs/tmp_ner.json') as f:
    lines=f.readlines()
    for line in lines:
        ner_data.append(json.loads(line.strip()))
for example in ner_data:
    answer=example['answer']
    question=example['question']
    
    topic_entity=answer.split('|||')[0].strip()
    if ('（' in topic_entity and '）' in topic_entity):
        topic_entity=topic_entity.split('（')[0]
    try:
        assert topic_entity in question
    except:
        print(example,topic_entity)
        raise Exception("check")
    ner_pred=example['ner_pred']
    if ner_pred==topic_entity:
        TP+=1
    else:
        if ner_pred not in question:
            FN+=1#is not a true entity
            FN_examples.append(example)
            continue
            
        ner_pred_rule1=rule1_for_find_ner(topic_entity=ner_pred,question=question,sub_map=sub_map,alias_map=alias_map)
        ner_pred_rule2=rule2_for_find_ner(topic_entity=ner_pred,question=question,sub_map=sub_map,alias_map=alias_map)
        if ner_pred_rule1 == topic_entity or ner_pred_rule2 == topic_entity:
            TP+=1
        else:
            if ner_pred in alias_map or ner_pred in sub_map:
                FP+=1#is a true entity in KG but not current example
                FP_examples.append(example)
            else:
                FN+=1#is not a true entity
                FN_examples.append(example)

In [149]:
print(TP,FP,FN)

1977 7 16


In [150]:
recall=TP/(TP+FN)
precision=TP/(TP+FP)
f1=2*recall*precision/(recall+precision)
print(recall,precision,f1)

0.9919719016557953 0.9964717741935484 0.9942167462911742


In [151]:
FN_examples

[{'id': 144,
  'question': '你知道甑字的偏旁是什么吗？',
  'answer': '甑 ||| 简体部首 ||| 瓦',
  'ner_pred': ''},
 {'id': 195,
  'question': '你什么时候参加的城市规划师考试的啊？',
  'answer': '城市规划师 ||| 考试时间 ||| 10 月中旬',
  'ner_pred': '城市规划师考试'},
 {'id': 293,
  'question': '艾德盖恩：平原镇屠夫资源是什么类型？',
  'answer': '艾德盖恩：平原镇屠夫 ||| 资源类型 ||| DVDRip',
  'ner_pred': '艾德盖恩：平原镇屠夫资源'},
 {'id': 353,
  'question': '第25届北京大学生电影节获得最佳男主角的详细资料有哪些？',
  'answer': '第25届北京大学生电影节 ||| 最佳男主角 ||| 张震',
  'ner_pred': '第25届北京大学生电影节获得最佳男主角'},
 {'id': 502,
  'question': '天使投资印张一共有几个？',
  'answer': '天使投资（2012年浙江人民出版社出版的图书） ||| 印张 ||| 19.5',
  'ner_pred': '天使投资印张'},
 {'id': 567,
  'question': '东北鼢鼠有什么天敌啊',
  'answer': '东北鼢鼠（哺乳纲动物） ||| 天敌 ||| 猫、猫头鹰和蛇等',
  'ner_pred': '东 北 鼠'},
 {'id': 659,
  'question': '于鋆炜的拦网高度是多少',
  'answer': '于鋆炜 ||| 拦网高度 ||| 300cm',
  'ner_pred': '于 炜'},
 {'id': 712,
  'question': '大佬们，《汉熹平石经残字集录》在哪个博物馆收藏的啊',
  'answer': '《汉熹平石经残字集录》 ||| 所属博物馆 ||| 洛阳金石文字博物馆',
  'ner_pred': '汉熹平石经残字集录'},
 {'id': 907,
  'question': '10·26朴正熙遇刺事件有哪些人员',
 

In [153]:
def rule1_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity为起始点，往右边找
    '''
    assert topic_entity in question
    start_idx=question.find(topic_entity)
    end_idx=start_idx+len(topic_entity)
    for i in range(end_idx,len(question)):
        if question[start_idx:i] in sub_map or question[start_idx:i] in alias_map:
            topic_entity=question[start_idx:i]
            break
    
    return topic_entity

def rule2_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity为终止点，往左边找
    '''
    assert topic_entity in question
    start_idx=question.find(topic_entity)
    end_idx=start_idx+len(topic_entity)
    for i in range(0,start_idx+1):
        if question[i:end_idx] in sub_map or question[i:end_idx] in alias_map:
            topic_entity=question[i:end_idx]
            break
    
    return topic_entity

def rule3_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity中心，从左到右删除字符
    '''
    assert topic_entity in question
    for i in range(len(topic_entity)):
        if (topic_entity[i:] in sub_map or topic_entity[i:] in alias_map):
            topic_entity=topic_entity[i:]
            break
    return topic_entity

def rule4_for_find_ner(topic_entity,question,sub_map,alias_map):
    '''
    以topic_entity中心，从右到左删除字符
    '''
    assert topic_entity in question
    for i in range(len(topic_entity),1):
        if topic_entity[:i] in sub_map or topic_entity[:i] in alias_map:
            topic_entity=topic_entity[:i]
            break
    return topic_entity


In [158]:
i=3
FN_examples[i]

{'id': 353,
 'question': '第25届北京大学生电影节获得最佳男主角的详细资料有哪些？',
 'answer': '第25届北京大学生电影节 ||| 最佳男主角 ||| 张震',
 'ner_pred': '第25届北京大学生电影节获得最佳男主角'}

In [155]:
topic_entity=FN_examples[i]['ner_pred']
question=FN_examples[i]['question']
print(topic_entity,question)

艾德盖恩：平原镇屠夫资源 艾德盖恩：平原镇屠夫资源是什么类型？


In [156]:
rule3_for_find_ner(topic_entity,question,sub_map,alias_map)

'资源'

In [157]:
rule4_for_find_ner(topic_entity,question,sub_map,alias_map)

'艾德盖恩：平原镇屠夫资源'

In [215]:
def read_data_nlpcc(data_path):
    with open(data_path) as f:
        lines=f.readlines()

    examples=[]
    for line in lines:
        examples.append(json.loads(line.strip()))
    
    print("The number examples: {} from {}".format(len(examples),data_path))
    return examples

data=read_data_nlpcc('../../data/nlpcc2018/test.txt')

The number examples: 9553 from ../../data/nlpcc2018/test.txt


In [216]:
data[0]

{'question': '你知道计算机应用基础这本书的作者是谁吗？', 'answer': '计算机应用基础 ||| 作者 ||| 秦婉，王蓉'}

In [217]:
data_type='nlpcc'
bad_examples=[]
left_examples=[]
for example in data:
    answer=example['answer']
    question=example['question']

    topic_entity=answer.split('|||')[0].strip()
    if ('（' in topic_entity and '）' in topic_entity):
        topic_entity=topic_entity.split('（')[0]

    try:
        assert topic_entity in question
        left_examples.append(example)
    except:
        #print(example,topic_entity)
        #raise Exception("check")
        bad_examples.append(example)

In [218]:
print(len(bad_examples),len(left_examples))

0 9553


In [197]:
bad_examples

[{'question': '万达的总部在哪',
  'answer': '万达广场 ||| 总部地点 ||| 重庆市南岸区江南大道8号（重庆万达）/重庆市万州区北滨路（万州万达）'},
 {'question': '郑州驱逐舰型号是什么有人知道吗', 'answer': '郑州号导弹驱逐舰 ||| 型号 ||| 国产052c型'},
 {'question': '笑傲江湖是什么语种的电视剧', 'answer': '《笑傲江湖》 ||| 语言版本 ||| 粤语、普通话'},
 {'question': '我想问一下多沦落猛龙队是啥时候建立的？', 'answer': '多伦多猛龙队 ||| 成立时间 ||| 1980年'},
 {'question': '梁氏从何起源？', 'answer': '梁氏起源 ||| 起源 ||| 自赢姓'},
 {'question': '类胡萝卜识别码是多少？', 'answer': '类胡萝卜素 ||| 识别 ||| 116-31-4'},
 {'question': '海地人是什么种族的？', 'answer': '海底人 ||| 种族 ||| 蛤蟆人'},
 {'question': '格兰仕kfr-32gw/dlp45-130的空调匹数是多少？',
  'answer': '格兰仕kfr-32gw/dlp45-130(2) ||| 空调匹数 ||| ：小1.5p'},
 {'question': '陶圆梦的品质如何？', 'answer': '陶园梦 ||| 品质 ||| 精选最上等的高原牦牛骨粉'},
 {'question': 'hp pavilion dv2304tx的显示屏是什么尺寸的？',
  'answer': 'hp pavilion dv2304tx(gc235pa) ||| 显示屏尺寸 ||| intel'},
 {'question': '请问史蒂芬霍金的著名成就是什么？',
  'answer': '史蒂芬·霍金 ||| 著名成就 ||| 霍金辐射 彭罗斯–霍金奇性定理（英语：penrose–hawking singularity theorems） 《时间简史：从大爆炸到黑洞》'},
 {'question': '请问einecs登录号是什么啊？', 'answer': '硫化钠 ||| einec

In [142]:
sub_map['迹象展览']

set()

In [159]:
sub_map['康熙字典']

set()

In [160]:
alias_map['康熙字典']

{'康熙字典（清代康熙时期的一种字典）'}

In [112]:
topic_entity

'宪法号博物馆'

In [113]:
ner_pred

'宪法号博物馆'

In [57]:
'agustin isunza' in 'Agustín Isunza有哪些合作导演'.lower()

False

In [75]:
zhconv.convert('委员會','zh-hans')

'委员会'

In [87]:
final_rule(question='Agustín Isunza有哪些合作导演',ner_pred='agustin isunza')

'Agustín Isunza'

In [103]:
data[:20]

[{'id': 0,
  'question': '你知道长江武汉航道局管多少个地方吗？',
  'answer': '长江武汉航道局 ||| 管辖 ||| 715.2公里',
  'ner_pred': '长江武汉航道局管'},
 {'id': 1,
  'question': '东瓯王发生的主要事件是什么？',
  'answer': '东瓯王 ||| 主要事件 ||| 抗秦反秦，助汉击楚。',
  'ner_pred': '东瓯王发'},
 {'id': 2,
  'question': '山西省铸造新工艺工程技术研究中心是什么的管辖的啊？',
  'answer': '山西省铸造新工艺工程技术研究中心 ||| 机构类型 ||| 研究开发机构',
  'ner_pred': '山西省铸造新工艺工程技术研究中心是'},
 {'id': 3,
  'question': '你知道谁是丘堤的儿媳吗',
  'answer': '丘堤 ||| 儿媳 ||| 籍虹',
  'ner_pred': '丘堤的'},
 {'id': 4,
  'question': '国民军参与过什么战争？',
  'answer': '国民军 ||| 参与战争 ||| 直奉战争、北伐战争、中原大战、抗日战争',
  'ner_pred': '国民军参'},
 {'id': 5,
  'question': '你知道诺基亚lumia1320的主屏尺寸是多少吗？',
  'answer': '诺基亚lumia1320 ||| 主屏尺寸 ||| 6英寸',
  'ner_pred': '诺基亚lumia1320的'},
 {'id': 6,
  'question': '你什么时候参加的执业兽医资格考试啊？',
  'answer': '执业兽医资格考试 ||| 考试时间 ||| 10月',
  'ner_pred': '执业兽医资格考试啊'},
 {'id': 7,
  'question': '你知道2007年度北京流行音乐典礼是哪种类型的奖项吗',
  'answer': '2007年度北京流行音乐典礼 ||| 奖项类型 ||| 音乐类奖项',
  'ner_pred': '2007年度北京流行音乐典礼是'},
 {'id': 8,
  'question': 'BOT的总价要多少钱',


In [67]:
re.findall('“(.*)”','请问“一带一路”铁路国际人才教育联盟是由哪个单位发起的？')

['一带一路']

In [69]:
'一带一路铁路国际人才教育联盟'.replace('一带一路','“一带一路”')

'“一带一路”铁路国际人才教育联盟'

In [ ]:
zhconv.

In [21]:
for example in data[:10]:
    text=example['question'].lower().replace('“','').replace('”','').replace('"','').replace('"','').replace('\n','').replace('\t','')
    text='{}'.format(text)+'。上个句子中哪些属于实体名词？_。'#'你知道泰山极顶的作家是谁啊？。上个句子中哪些属于实体名词？_。'
    model_inputs=tokenizer([text],return_tensors='pt')
    input_ids=model_inputs['input_ids'].to(device)
    attention_mask=model_inputs['attention_mask'].to(device)
    outputs = model.generate(input_ids=input_ids,
                             attention_mask=attention_mask,
                             num_beams=4,
                             min_length=1,
                             max_length=30,
                             early_stopping=True,)#tensor([[ 102,  101, 3805, 2255, 3353, 7553,  102]], device='cuda:0')
    pred=tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=True).strip()#'泰 山 极 顶'
    

In [22]:
outputs

tensor([[ 102,  101, 3805, 2255, 3353, 7553,  102]], device='cuda:0')

In [27]:
text

'你知道泰山极顶的作家是谁啊？。上个句子中哪些属于实体名词？_。'

In [24]:
tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_spaces=True).strip()

'泰 山 极 顶'

In [26]:
data[9]

{'id': 9, 'question': '你知道泰山极顶的作家是谁啊？', 'answer': '泰山极顶 ||| 作家 ||| 杨朔'}

In [8]:
text

'宪法号博物馆位于哪里？。上个句子中哪些属于实体名词？_。'

In [13]:
tokenizer([text],return_tensors='pt')

{'input_ids': tensor([[ 101, 2150, 3791, 1384, 1300, 4289, 7667,  855,  754, 1525, 7027, 8043,
          511,  677,  702, 1368, 2094,  704, 1525,  763, 2247,  754, 2141,  860,
         1399, 6404, 8043,  142,  511,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

In [77]:
examples=[]
with open('/home/xhsun/Desktop/graduate_models/Section4/kgclue/finegrained/ner-predictions.json') as f:
    lines=f.readlines()
for line in lines:
    examples.append(json.loads(line.strip()))

In [78]:
examples[:3]

[{'question': '你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于组织机构？',
  'answer': ['空实体'],
  'predict': '长江武汉航道局'},
 {'question': '你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于人名？',
  'answer': ['空实体'],
  'predict': '空实体'},
 {'question': '你知道长江武汉航道局管多少个地方吗？。这个句子中有哪些属于音乐？',
  'answer': ['空实体'],
  'predict': '风铃'}]

In [79]:
def normalize_string(token):
    token=token.lower()
    token=token.replace('“','').replace('”','').replace('《','').replace('》','').replace('。','').replace('-','')
    token=zhconv.convert(token,'zh-hans')
    return token

In [80]:
print(len(examples))
res={}
for e  in examples:
    q=e['question'].split('。这个句子中有哪些')[0]
    q=normalize_string(q)
    has_en=False
    for char in q:
        if char in string.ascii_letters:
            has_en=True
            break
    if has_en:
        continue
        
    if q not in res:
        res[q]={"answer":[],'predict':set()}
    answer=e['answer']
    assert len(answer)==1
    answer=answer[0]
    answer=normalize_string(answer)
    predict=e['predict']
    predict=normalize_string(predict)
    assert type(predict)==str
    if answer!='空实体':
        res[q]['answer'].append(answer)
    if predict!='空实体':
        if predict in q and predict!='':
            res[q]['predict'].add(predict)
        
print(len(list(res.keys())))

22000
1584


In [81]:
'乌戈·拉斐尔·查韦斯·弗里斯'

'乌戈·拉斐尔·查韦斯·弗里斯'

In [82]:
res

{'你知道长江武汉航道局管多少个地方吗？': {'answer': ['长江武汉航道局'], 'predict': {'长江武汉航道局'}},
 '东瓯王发生的主要事件是什么？': {'answer': ['东瓯王'], 'predict': {'东瓯王'}},
 '山西省铸造新工艺工程技术研究中心是什么的管辖的啊？': {'answer': ['山西省铸造新工艺工程技术研究中心'],
  'predict': set()},
 '你知道谁是丘堤的儿媳吗': {'answer': ['丘堤'], 'predict': {'丘堤'}},
 '国民军参与过什么战争？': {'answer': ['国民军'], 'predict': {'国民军'}},
 '你什么时候参加的执业兽医资格考试啊？': {'answer': ['执业兽医资格考试'], 'predict': {'执业兽医资格考试'}},
 '你知道2007年度北京流行音乐典礼是哪种类型的奖项吗': {'answer': ['2007年度北京流行音乐典礼'],
  'predict': set()},
 '你知道泰山极顶的作家是谁啊？': {'answer': ['泰山极顶'], 'predict': {'泰山极顶'}},
 '我想知道法国斗牛犬有什么特点？': {'answer': ['法国斗牛犬'], 'predict': {'法国斗牛犬'}},
 '请问七朵水仙花的作者是谁？': {'answer': ['七朵水仙花'], 'predict': {'七朵水仙花'}},
 '你知道皖系军阀参与了什么战争？': {'answer': ['皖系军阀'], 'predict': {'皖系军阀'}},
 '信号与系统分析什么时候印刷的？': {'answer': ['信号与系统分析'], 'predict': {'信号与系统分析'}},
 '请问设计成本控制是什么意思？': {'answer': ['设计成本控制'], 'predict': {'设计成本控制'}},
 '广东省预防青少年犯罪研究会由哪个单位发起？': {'answer': ['广东省预防青少年犯罪研究会'], 'predict': set()},
 '义安公主有侄女吗，叫什么啊': {'answer': ['义安公主'], 'predict': {'

In [83]:
number_span_total=0
number_span_right=0
number_span_wrong=0

for q,values in res.items():
    answer=values['answer']
    assert len(answer)==1
    number_span_total+=1
    answer=answer[0]
    predict=list(values['predict'])
    
    for p in predict:
        if p==answer:
            number_span_right+=1
        else:
            number_span_wrong+=1

In [84]:
recall=(number_span_right)/(number_span_total)
precision=(number_span_right)/(number_span_right+number_span_wrong)
f1=2*precision*recall/(precision+recall)

In [85]:
print(recall,precision,f1)

0.8434343434343434 0.9577060931899641 0.8969452836522323


In [86]:
number_span_total

1584

In [87]:
number_span_right

1336

In [88]:
number_span_wrong

59

In [38]:
# data=[]
# import json
# with open('/home/xhsun/Desktop/graduate_models/Section4/nlpcc/mixture/ner-predictions.json') as f:
#     lines=f.readlines()
#     for line in lines:
#         data.append(json.loads(line.strip()))
        
data=[]
import json
with open('/home/xhsun/Desktop/graduate_models/Section4/kgclue/mixture-predictions.json') as f:
    lines=f.readlines()
    for line in lines:
        data.append(json.loads(line.strip()))        

In [39]:
len(data)

6674

In [40]:
data[0]

{'question': '传次郎喜欢吃什么。实体名词是：传次郎。这个句子的意图与下列哪一个关系最相似？_。 （0）性别，（1）别名，（2）喜爱的食物，（3）初次登场，（4）登场作品，（5）外文名，（6）配音，（7）身份，（8）身高，（9）不匹配，（10）中文名',
 'answer': ['喜爱的食物'],
 'predict': '喜爱的食物'}

In [41]:
wrong_examples=[]
only_cn_examples=[]
import string
for example in data:
    question=example['question'].lower()
    en=False
    for char in question:
        if char in string.ascii_letters+"-"+"0123456789":
            en=True
            break
    if en==False:
        only_cn_examples.append(example)
    predict=example['predict']
    if predict not in question:
        wrong_examples.append(example)

In [42]:
len(only_cn_examples)

1387

In [43]:
len(data)

6674

In [44]:
wrong_examples=[]

for example in only_cn_examples:
    question=example['question'].lower()
    predict=example['predict']
    if predict not in question:
        wrong_examples.append(example)

In [45]:
len(wrong_examples)

7

In [46]:
(len(only_cn_examples)-len(wrong_examples))/len(only_cn_examples)

0.9949531362653208

In [47]:
wrong_examples

[{'question': '威海—青岛高速公路经过哪些线路？。上个句子中哪些属于实体名词？_。',
  'answer': ['威海—青岛高速公路'],
  'predict': '威海青岛高速公路'},
 {'question': '东北鼢鼠有什么天敌啊。上个句子中哪些属于实体名词？_。',
  'answer': ['东北鼢鼠'],
  'predict': '东北鼠'},
 {'question': '东海岛—雷州高速公路限速多少。上个句子中哪些属于实体名词？_。',
  'answer': ['东海岛—雷州高速公路'],
  'predict': '东海岛雷州高速公路'},
 {'question': '于鋆炜的拦网高度是多少。上个句子中哪些属于实体名词？_。',
  'answer': ['于鋆炜'],
  'predict': '于炜'},
 {'question': '你知道东营—深圳公路的沿途有哪些城市吗。上个句子中哪些属于实体名词？_。',
  'answer': ['东营—深圳公路'],
  'predict': '东营深圳公路'},
 {'question': '中国共产党中央委员会政法委员会属于什么机构你知道吗？。上个句子中哪些属于实体名词？_。',
  'answer': ['中国共产党中央委员会政法委员会'],
  'predict': '中国共产党中央委員会政法委员会'},
 {'question': '烟台—海阳高速公路是什么时候开始的？。上个句子中哪些属于实体名词？_。',
  'answer': ['烟台—海阳高速公路'],
  'predict': '烟台海阳高速公路'}]